In [24]:
def get_word_count(filename):
  with open(filename, 'r') as f:
    # Read the entire file content
    content = f.read()

    # Split the content into words, removing whitespaces
    words = content.split()

    return len(words)

# Example usage
filename = "./raw_data/combined_unlabeled_commentary_clean.txt"  # Replace with your actual filename
word_count = get_word_count(filename)
print(f"The file '{filename}' has {word_count} words.")
# print(f"The file '{filename2}' has {word_count2} words.")

The file './raw_data/combined_unlabeled_commentary_clean.txt' has 3962 words.


In [43]:
def split_sentences(input_file, output_file):
  """
  Splits a text file into separate lines for each sentence.

  Args:
    input_file (str): Path to the input text file.
    output_file (str): Path to the output file where the split sentences will be written.
  """
  sentences = []
  current_sentence = ""
  with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
    for line in infile:
      # Add line to current sentence (excluding trailing newline)
      current_sentence += line.strip()

      # Check for sentence delimiters (e.g., ., ?, !) and write complete sentences
      for delimiter in [".", "?", "!"]:
        if delimiter in current_sentence:
          if (len(sentences) < 100 and len(current_sentence) > 8):
            sentences.append(current_sentence.split(delimiter)[0] + delimiter + "\n")
          current_sentence = current_sentence.split(delimiter)[1].strip()

      # Write any remaining sentence at the end of the file
      # if current_sentence:
      #   sentences.append(current_sentence)
    outfile.writelines(sentences)

# Example usage:
input_file = "./raw_data/combined_unlabeled_commentary.txt"
output_file = "./raw_data/combined_unlabeled_commentary_clean.txt"  # Output file with split sentences

split_sentences(input_file, output_file)

print(f"Text file split into sentences and saved to {output_file}")

Text file split into sentences and saved to ./raw_data/combined_unlabeled_commentary_clean.txt


In [42]:
word_count = get_word_count(output_file)
word_count

# dataset

4061

In [263]:
from transformers import BertTokenizer, BertForMaskedLM, DataCollatorForLanguageModeling
from datasets import load_dataset
import torch

# Define paths and hyperparameters
model_name = "bert-base-uncased"  # Pre-trained BERT model
text_file = "./raw_data/combined_unlabeled_commentary_clean.txt"  # Path to your text file
batch_size = 8  # Adjust this based on your GPU memory
max_len = 512  # Maximum sequence length
learning_rate = 1e-5

# Check for GPU availability
device = torch.device("mps")

# Load tokenizer and model on chosen device
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [264]:
# Define function to mask tokens
def mask_sentence(sentence):
  """Masks 15% of the tokens in a sentence."""
  masked_sentence = tokenizer(sentence["text"], return_tensors="pt", truncation=True, padding=True)
  masked_sentence = masked_sentence.to(device)  # Move data to device

  input_ids = masked_sentence["input_ids"]
  attention_mask = masked_sentence["attention_mask"]
  labels = input_ids.clone()

  # Randomly mask 15% of tokens
  probability_mask = 0.15
  mask_indices = torch.bernoulli(torch.ones_like(input_ids) * probability_mask).bool()
  labels[~mask_indices] = -100  # Set padding tokens to -100

  # Replace masked tokens with [MASK]
  input_ids[mask_indices] = tokenizer.mask_token_id

  return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}


In [267]:
# Load data from text file
dataset = load_dataset("text", data_files={"train": text_file}, split="train")
train_data = dataset.map(mask_sentence, batched=True)
print(type(train_data))
train_data.set_format("torch")

print(type(train_data))
print(train_data)
print(type(train_data))
print(train_data)
print(type(train_data[0]))

train_loader = DataLoader(train_data, batch_size=5, shuffle=True)
num_batches = len(train_dataloader)
print(f"Number of batches: {num_batches}")

<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>
Dataset({
    features: ['text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 100
})
<class 'datasets.arrow_dataset.Dataset'>
Dataset({
    features: ['text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 100
})
<class 'dict'>
Number of batches: 13


In [ ]:
for epoch in range(10):  # Adjust number of epochs
  for batch in train_loader:
    # Move data to device
    #batch.to(device)
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    labels = batch["labels"].to(device)
    #print(batch["text"])

    # Forward pass
    outputs = model(input_ids= input_ids, attention_mask=attention_mask, labels=labels)
    loss = outputs.loss

    # Backward pass and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print training progress (optional)
    print(f"Epoch: {epoch+1}, Loss: {loss}")

# Save the fine-tuned model (optional)
model.save_pretrained("masked_lm_model")

Epoch: 1, Loss: 8.6705322265625
Epoch: 1, Loss: 9.833917617797852
Epoch: 1, Loss: 11.892194747924805
Epoch: 1, Loss: 10.5023193359375
Epoch: 1, Loss: 9.986804008483887
Epoch: 1, Loss: 7.155943393707275
Epoch: 1, Loss: 8.5454683303833
Epoch: 1, Loss: 8.85640811920166
Epoch: 1, Loss: 8.859716415405273
Epoch: 1, Loss: 7.355210781097412
Epoch: 1, Loss: 10.656970977783203
Epoch: 1, Loss: 9.582688331604004
Epoch: 1, Loss: 8.176599502563477
Epoch: 1, Loss: 7.87153434753418


<class 'datasets.arrow_dataset.Dataset'>
<class 'list'>


Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Dataset({
    features: ['a', 'b'],
    num_rows: 3
})
